In [2]:
try:
    import pandas as pd
    import numpy as np
    import seaborn as sns
    sns.set_style('white')
    sns.set_context('talk')
    import matplotlib.pyplot as plt
    print('All packages installed successfuly')
except Exception as e:
    print(f'Exception: {e}')

All packages installed successfuly


### The cookie problem
* Suppose we have two bowls of cookies, 
    * Bowl_1 contains 30 vanilla and 10 chocolate cookies
    * Bowl_2 contains 20 vanilla and 20 chocolate cookies
* A person chooses one bowl at random, and random picked up cookie turns out to be vanilla cookie.
* What is the probability that chosen Bowl is Bowl_1?

 Assume that there was an equal chance of choosing either bowl and an equal chance of choosing any cookie in the bowl.

In [11]:
number_of_bowls = 2
bowl_qty_list = [40,40]
vanilla_bowl_list = [30, 20]
chocolate_bowl_list = [30, 20]

In [10]:
prior_probabilities = [1/(number_of_bowls) for num in range(number_of_bowls)]
prior_probabilities

[0.5, 0.5]

In [12]:
vanilla_unnormalized_probs = [van/bowl_qty_list[index] for index, van in enumerate(vanilla_bowl_list)] 
vanilla_unnormalized_probs

[0.75, 0.5]

In [14]:
vanilla_normalized_sum  = sum(vanilla_unnormalized_probs)/ len(vanilla_unnormalized_probs)
vanilla_normalized_sum_list = [vanilla_normalized_sum for van in range(len(vanilla_unnormalized_probs))]
vanilla_normalized_sum_list

[0.625, 0.625]

In [16]:
new_df = pd.DataFrame()
new_df['prior_probabilities'] = prior_probabilities
new_df['vanilla_unnormalized_probs'] = vanilla_unnormalized_probs
new_df['vanilla_normalized_sum_list'] = vanilla_normalized_sum_list
new_df.index = 'Bowl_1', 'Bowl_2'
new_df

,prior_probabilities,vanilla_unnormalized_probs,vanilla_normalized_sum_list
Bowl_1,0.5,0.75,0.625
Bowl_2,0.5,0.50,0.625


In [17]:
new_df['posterior'] = (new_df['prior_probabilities'] * new_df['vanilla_unnormalized_probs']) / new_df['vanilla_normalized_sum_list']
new_df

,prior_probabilities,vanilla_unnormalized_probs,vanilla_normalized_sum_list,posterior
Bowl_1,0.5,0.75,0.625,0.6
Bowl_2,0.5,0.50,0.625,0.4
